In [1]:
from github import Github
import getpass
import time
import pandas as pd
import csv
import langdetect
import base64
import os
import json
from datetime import datetime

g = Github(getpass.getpass("Enter access token:"), per_page=100)

Enter access token: ········


Retrieves the release data (if there are any) for repositories with readme files. (This condition might be removed to collect releases of repositories without readme files.)

In [32]:
languages = ["Java", "Python"]

if not os.path.exists("releases"):
    os.makedirs("releases")

for language in languages:

    candidate_repos = pd.read_csv(f"candidate_repos_{language.lower()}.csv", delimiter=";")
    print(f"Retrieving the releases for {candidate_repos.shape[0]} repositories in {language}...")
    
    collected = 0
    
    for row_id, row in candidate_repos.iterrows():
        readme_file = f"readmes/{row.OWNER}_{row.NAME}_readme.md"
        release_file = f"releases/{row.OWNER}_{row.NAME}_releases.jsonl"
        if not os.path.exists(readme_file) or os.path.exists(release_file):
            continue
            
        repo = g.get_repo(f"{row.OWNER}/{row.NAME}")
        time.sleep(0.5)
        try:
            releases = repo.get_releases()
            
            if releases.totalCount < 1:
                continue
            
            with open(release_file, "w", encoding="utf-8") as file:
                for release in releases:
                    file.write(json.dumps(release.raw_data) + "\n")
                    time.sleep(1)
                    
            collected += 1
        except Exception as e:
            print(f"{row.OWNER}/{row.NAME}: {e}")
            if os.path.exists(release_file):
                os.remove(release_file) 
            continue
                
    print(f"{collected} repositories' releases were collected.")

Retrieving the releases for 1881 repositories in Java...
358 repositories' releases were collected.
Retrieving the releases for 2123 repositories in Python...
1219 repositories' releases were collected.
